In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

%load_ext watermark
%watermark -a 'stekaiser' -d -t -v -p numpy,pandas,matplotlib,keras,tensorflow -g

Using TensorFlow backend.


stekaiser 2016-11-30 10:38:42 

CPython 3.5.2
IPython 5.1.0

numpy 1.11.2
pandas 0.19.1
matplotlib 1.5.3
keras 1.1.1
tensorflow 0.11.0
Git hash: HEAD


In [2]:
import os
import sys

project_dir = os.path.join(os.getcwd(), os.pardir)
sys.path.insert(0, project_dir)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import zipfile
from dotenv import find_dotenv, load_dotenv

# show all columns
pd.set_option('display.max_columns', None)

In [3]:
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Convolution1D
from keras.layers import MaxPooling1D,Flatten
from keras.layers import GRU
from keras.layers import Dropout
from keras.regularizers import l2, activity_l2
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import load_model
from keras.callbacks import TensorBoard,ModelCheckpoint,EarlyStopping,Callback
# fix random seed for reproducibility
np.random.seed(7)

sys.setrecursionlimit(50000)

In [4]:
import itertools
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics

In [5]:
batch_size = 64
nTimesteps = 128

In [6]:
X = pd.read_csv('https://dl.dropboxusercontent.com/u/558313/keras-js/test.csv', header=None, nrows=nTimesteps*batch_size)
X.head()

,0,1,2,3,4,5
0,-0.675415,0.119995,0.767578,-0.095274,-0.266768,-0.213415
1,-0.678223,0.120972,0.772949,-0.323933,-0.400152,-0.571646
2,-0.679810,0.117310,0.770142,-0.434451,-0.457317,-0.567835
3,-0.681519,0.115234,0.767822,-0.125762,-0.514482,-0.522104
4,-0.680298,0.113770,0.773804,-0.053354,-0.327744,-0.472561


In [7]:
X = X.as_matrix()

In [8]:
X.shape

(8192, 6)

In [9]:
X = np.reshape(X, (-1, nTimesteps, 6))
X.shape

(64, 128, 6)

In [10]:
target = np.hstack((np.ones((1,batch_size/2)), np.zeros((1,batch_size/2))))
target = np.transpose(target)

/home/steffen/bin/anaconda3/envs/equestic/lib/python3.5/site-packages/numpy/core/numeric.py:190: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  a = empty(shape, dtype, order)
/home/steffen/bin/anaconda3/envs/equestic/lib/python3.5/site-packages/ipykernel/__main__.py:1: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':


In [11]:
Y = np.column_stack((target, np.flipud(target)))

In [12]:
Y.shape

(64, 2)

In [13]:
n_hidden = 64

model = Sequential()
model.add(LSTM(n_hidden,
               return_sequences=True,
               batch_input_shape=(batch_size, nTimesteps, X.shape[2])
              ))
model.add(Dropout(0.2))
model.add(LSTM(n_hidden))
model.add(Dropout(0.2))
model.add(Dense(2, activation='softmax'))

model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [14]:
print(model.summary())

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lstm_1 (LSTM)                    (64, 128, 64)         18176       lstm_input_1[0][0]               
____________________________________________________________________________________________________
dropout_1 (Dropout)              (64, 128, 64)         0           lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (64, 64)              33024       dropout_1[0][0]                  
____________________________________________________________________________________________________
dropout_2 (Dropout)              (64, 64)              0           lstm_2[0][0]                     
___________________________________________________________________________________________

In [15]:
history = model.fit(X,Y,batch_size=batch_size)

Epoch 1/10
64/64 [==============================] - 1s - loss: 0.7442 - acc: 0.4531
Epoch 2/10
64/64 [==============================] - 0s - loss: 0.5825 - acc: 0.6875
Epoch 3/10
64/64 [==============================] - 0s - loss: 0.4752 - acc: 0.8281
Epoch 4/10
64/64 [==============================] - 0s - loss: 0.4169 - acc: 0.8438
Epoch 5/10
64/64 [==============================] - 0s - loss: 0.3731 - acc: 0.9062
Epoch 6/10
64/64 [==============================] - 0s - loss: 0.3300 - acc: 0.9219
Epoch 7/10
64/64 [==============================] - 0s - loss: 0.2909 - acc: 0.9219
Epoch 8/10
64/64 [==============================] - 0s - loss: 0.2626 - acc: 0.9219
Epoch 9/10
64/64 [==============================] - 0s - loss: 0.2512 - acc: 0.9375
Epoch 10/10
64/64 [==============================] - 0s - loss: 0.2087 - acc: 0.9688


In [16]:
model.save_weights('model.hdf5')
with open('model.json', 'w') as f:
    f.write(model.to_json())

In [17]:
!python encoder.py model.hdf5